In [193]:
import pandas as pd
import os, glob
from owlready2 import *

import owlready2
print(owlready2.VERSION) ## Bugs when > 0.40 but works file at owlready2==0.40 

0.40


In [2]:
onto = get_ontology("kg/pbn_t3_5.owl").load()

In [3]:
ALL = {}
for k in onto.classes():
    K = str(k).split(".")[-1]
    if K == "Risk":
        for i in k.instances():
            I = str(i).split(".")[-1]
            ALL[I[5:]] = {"mitig":[],"subject":[],"owner":[],"category":[]}
            for p in i.get_properties():
                #print("---",p)
                P = str(p).split(".")[-1]
                OBJ = "onto."+str(I)+"."+str(P)
                B = eval(OBJ)
                #print(OBJ,I,P,B)
                # has_RiskSubject, has_RiskOwner
                for b in B:
                    if P == "has_RiskSubject":
                        ALL[I[5:]]["subject"].append(str(b).split(".")[-1][5:])
                    elif P == "has_RiskOwner":
                        ALL[I[5:]]["owner"].append(str(b).split(".")[-1][5:])
                    elif P == "has_RiskMitigation":
                        ALL[I[5:]]["mitig"].append(str(b).split(".")[-1][5:])
                    elif P == "has_Category":
                        ALL[I[5:]]["category"].append(str(b).split(".")[-1][5:])
                        
dfRisk = pd.DataFrame.from_dict(ALL,orient='index' ).reset_index()
dfRisk.columns = ["Risk","mitig","subject","owner","category"]
for col in dfRisk.columns:
    dfRisk = dfRisk.explode(column=col)
print(len(dfRisk))
dfRisk.head(20)



40532


,Risk,mitig,subject,owner,category
0,Risk_0,Mitigation_0,NaN,NaN,PBNCategory_50
1,Risk_828,Mitigation_416,NaN,NaN,PBNCategory_50
2,Risk_459,Mitigation_649,NaN,NaN,PBNCategory_50
3,Risk_642,Mitigation_988,Stakeholder_68,Stakeholder_394,PBNCategory_50
3,Risk_642,Mitigation_988,Stakeholder_68,Stakeholder_46,PBNCategory_50
3,Risk_642,Mitigation_988,Stakeholder_68,Stakeholder_47,PBNCategory_50
3,Risk_642,Mitigation_988,Stakeholder_68,Stakeholder_73,PBNCategory_50
3,Risk_642,Mitigation_988,Stakeholder_68,Stakeholder_269,PBNCategory_50
3,Risk_642,Mitigation_988,Stakeholder_9,Stakeholder_394,PBNCategory_50
3,Risk_642,Mitigation_988,Stakeholder_9,Stakeholder_46,PBNCategory_50


In [4]:
eval("onto."+"PBN__PBNCategory_70").label

['The pandemic has led to a decrease in in-person surgical education, loss of control, temporary panic from cruise ship docking, limited social distancing, and decreased morale.']

In [5]:
## MITIG
ALL = {}
for k in onto.classes():
    K = str(k).split(".")[-1]
    if K == "Mitigation":
        for i in k.instances():
            I = str(i).split(".")[-1]
            ALL[I[5:]] = {"benefit":[],"technology":[]}
            for p in i.get_properties():
                P = str(p).split(".")[-1]
                OBJ = "onto."+str(I)+"."+str(P)
                B = eval(OBJ)
                #print(OBJ,I,P,B)
                # has_MitigationBenefit , has_MitigationTechnology
                for b in B:
                    if P == "has_MitigationBenefit":
                        ALL[I[5:]]["benefit"].append(str(b).split(".")[-1][5:])
                    elif P == "has_MitigationTechnology":
                        ALL[I[5:]]["technology"].append(str(b).split(".")[-1][5:])
print(len(ALL))
dfMitig = pd.DataFrame.from_dict(ALL,orient='index' ).reset_index()
dfMitig.columns = ["mitig","benefit","tech"]
for col in dfMitig.columns:
    dfMitig = dfMitig.explode(column=col)
print(len(dfMitig))
dfMitig.head(3)

5295
7314


,mitig,benefit,tech
0,Mitigation_0,Benef_0,NaN
0,Mitigation_0,Benef_1,NaN
0,Mitigation_0,Benef_2,NaN


In [6]:
# 'Stakeholder' 'Risk' 'Mitigation' 'Technology'
# Risks -> Mitig
# Risks -> Affected, Owner
# Risks -> Techs

In [7]:
ALL = []
for k in onto.classes():
    K = str(k).split(".")[-1]
    if K == "PBNCategory":
        for i in k.instances():
            I = str(i).split(".")[-1]
            for p in i.get_properties():
                P = str(p).split(".")[-1]
                OBJ = "onto."+str(I)+"."+str(P)
                B = eval(OBJ)
                print(OBJ,I,P,B)
                if P == "has_CategoryElements":
                    LABEL = eval("onto."+str(I)+".label")[0]
                    #print(B[0].label[0],str(B[0]).split(".")[-1])
                    for b in B:
                        ALL.append([I,LABEL,str(b).split(".")[-1],b.label[0]])
print(len(ALL))


onto.PBN__PBNCategory_0.has_CategoryElements PBN__PBNCategory_0 has_CategoryElements [pbn_t3_5.PBN__Benef_0, pbn_t3_5.PBN__Benef_2, pbn_t3_5.PBN__Benef_181, pbn_t3_5.PBN__Benef_218, pbn_t3_5.PBN__Benef_414, pbn_t3_5.PBN__Benef_428, pbn_t3_5.PBN__Benef_444, pbn_t3_5.PBN__Benef_1100, pbn_t3_5.PBN__Benef_499, pbn_t3_5.PBN__Benef_525, pbn_t3_5.PBN__Benef_658, pbn_t3_5.PBN__Benef_661, pbn_t3_5.PBN__Benef_719, pbn_t3_5.PBN__Benef_1098, pbn_t3_5.PBN__Benef_863, pbn_t3_5.PBN__Benef_876, pbn_t3_5.PBN__Benef_879, pbn_t3_5.PBN__Benef_881, pbn_t3_5.PBN__Benef_940, pbn_t3_5.PBN__Benef_965, pbn_t3_5.PBN__Benef_991, pbn_t3_5.PBN__Benef_1024, pbn_t3_5.PBN__Benef_1146, pbn_t3_5.PBN__Benef_1183, pbn_t3_5.PBN__Benef_1203, pbn_t3_5.PBN__Benef_1207, pbn_t3_5.PBN__Benef_1306, pbn_t3_5.PBN__Benef_1314, pbn_t3_5.PBN__Benef_1387, pbn_t3_5.PBN__Benef_1388, pbn_t3_5.PBN__Benef_1393, pbn_t3_5.PBN__Benef_1397]
onto.PBN__PBNCategory_0.label PBN__PBNCategory_0 label ['Enhanced identification, prompt response, and ef

onto.PBN__PBNCategory_265.has_CategoryElements PBN__PBNCategory_265 has_CategoryElements [pbn_t3_5.PBN__Stakeholder_2205, pbn_t3_5.PBN__Stakeholder_796, pbn_t3_5.PBN__Stakeholder_2167, pbn_t3_5.PBN__Stakeholder_661, pbn_t3_5.PBN__Stakeholder_17, pbn_t3_5.PBN__Stakeholder_111, pbn_t3_5.PBN__Stakeholder_1202, pbn_t3_5.PBN__Stakeholder_479, pbn_t3_5.PBN__Stakeholder_2500, pbn_t3_5.PBN__Stakeholder_2098, pbn_t3_5.PBN__Stakeholder_23, pbn_t3_5.PBN__Stakeholder_2175, pbn_t3_5.PBN__Stakeholder_853, pbn_t3_5.PBN__Stakeholder_1142, pbn_t3_5.PBN__Stakeholder_2221, pbn_t3_5.PBN__Stakeholder_1577, pbn_t3_5.PBN__Stakeholder_2146, pbn_t3_5.PBN__Stakeholder_40, pbn_t3_5.PBN__Stakeholder_612, pbn_t3_5.PBN__Stakeholder_2327, pbn_t3_5.PBN__Stakeholder_71, pbn_t3_5.PBN__Stakeholder_1538, pbn_t3_5.PBN__Stakeholder_2370, pbn_t3_5.PBN__Stakeholder_2340, pbn_t3_5.PBN__Stakeholder_1020, pbn_t3_5.PBN__Stakeholder_1336, pbn_t3_5.PBN__Stakeholder_2286, pbn_t3_5.PBN__Stakeholder_1520, pbn_t3_5.PBN__Stakeholder_10

onto.PBN__PBNCategory_130.label PBN__PBNCategory_130 label ['Promote self-cleaning spaces, implement touchless technologies, and use natural light to establish safer environments.']
onto.PBN__PBNCategory_22.has_CategoryElements PBN__PBNCategory_22 has_CategoryElements [pbn_t3_5.PBN__Benef_39, pbn_t3_5.PBN__Benef_70, pbn_t3_5.PBN__Benef_71, pbn_t3_5.PBN__Benef_128, pbn_t3_5.PBN__Benef_220, pbn_t3_5.PBN__Benef_355, pbn_t3_5.PBN__Benef_396, pbn_t3_5.PBN__Benef_397, pbn_t3_5.PBN__Benef_1337, pbn_t3_5.PBN__Benef_506, pbn_t3_5.PBN__Benef_515, pbn_t3_5.PBN__Benef_536, pbn_t3_5.PBN__Benef_539, pbn_t3_5.PBN__Benef_779, pbn_t3_5.PBN__Benef_817, pbn_t3_5.PBN__Benef_953, pbn_t3_5.PBN__Benef_1047, pbn_t3_5.PBN__Benef_1049, pbn_t3_5.PBN__Benef_1050, pbn_t3_5.PBN__Benef_1053, pbn_t3_5.PBN__Benef_1088, pbn_t3_5.PBN__Benef_1090, pbn_t3_5.PBN__Benef_1109, pbn_t3_5.PBN__Benef_1134, pbn_t3_5.PBN__Benef_1136, pbn_t3_5.PBN__Benef_1197, pbn_t3_5.PBN__Benef_1198, pbn_t3_5.PBN__Benef_1227, pbn_t3_5.PBN__Benef_

onto.PBN__PBNCategory_158.label PBN__PBNCategory_158 label ['Advancements in various technological domains facilitate touchless, remote, and efficient ways of communication, work, and data management.']
onto.PBN__PBNCategory_170.has_CategoryElements PBN__PBNCategory_170 has_CategoryElements [pbn_t3_5.PBN__Technology_2608, pbn_t3_5.PBN__Technology_2613, pbn_t3_5.PBN__Technology_335, pbn_t3_5.PBN__Technology_3043, pbn_t3_5.PBN__Technology_3460, pbn_t3_5.PBN__Technology_3575, pbn_t3_5.PBN__Technology_2750, pbn_t3_5.PBN__Technology_404, pbn_t3_5.PBN__Technology_426, pbn_t3_5.PBN__Technology_3382, pbn_t3_5.PBN__Technology_2402, pbn_t3_5.PBN__Technology_2799, pbn_t3_5.PBN__Technology_6, pbn_t3_5.PBN__Technology_3871, pbn_t3_5.PBN__Technology_2357, pbn_t3_5.PBN__Technology_1077, pbn_t3_5.PBN__Technology_123, pbn_t3_5.PBN__Technology_2912, pbn_t3_5.PBN__Technology_2945, pbn_t3_5.PBN__Technology_419, pbn_t3_5.PBN__Technology_809, pbn_t3_5.PBN__Technology_1365, pbn_t3_5.PBN__Technology_2163, pbn

onto.PBN__PBNCategory_178.label PBN__PBNCategory_178 label ['Enhancement of knowledge and skills through resources, initiatives, programs, and proper utilization of personnel for various purposes.']
onto.PBN__PBNCategory_189.has_CategoryElements PBN__PBNCategory_189 has_CategoryElements [pbn_t3_5.PBN__Technology_1377, pbn_t3_5.PBN__Technology_48, pbn_t3_5.PBN__Technology_971, pbn_t3_5.PBN__Technology_2760, pbn_t3_5.PBN__Technology_146, pbn_t3_5.PBN__Technology_594, pbn_t3_5.PBN__Technology_1390, pbn_t3_5.PBN__Technology_1008, pbn_t3_5.PBN__Technology_2127, pbn_t3_5.PBN__Technology_2126, pbn_t3_5.PBN__Technology_50, pbn_t3_5.PBN__Technology_1030, pbn_t3_5.PBN__Technology_1871, pbn_t3_5.PBN__Technology_661, pbn_t3_5.PBN__Technology_2177, pbn_t3_5.PBN__Technology_3439, pbn_t3_5.PBN__Technology_2762, pbn_t3_5.PBN__Technology_120, pbn_t3_5.PBN__Technology_1982, pbn_t3_5.PBN__Technology_2993, pbn_t3_5.PBN__Technology_1617, pbn_t3_5.PBN__Technology_1616, pbn_t3_5.PBN__Technology_2178, pbn_t3_

In [8]:
from OAI import *

%load_ext autoreload
%autoreload 2

In [9]:
df = pd.DataFrame(ALL)
df.columns = ["Category","CategoryLabel","Item","Label"]
df.Category = df.Category.apply(lambda x: x[5:])
df.Item = df.Item.apply(lambda x: x[5:])
df["TYPE"] = df.Item.apply(lambda x: "_".join(x.split("_")[:-1]))
df["TYPE2"] = df.Item.apply(lambda x: x.split("_")[-2])
df.sample(3)

,Category,CategoryLabel,Item,Label,TYPE,TYPE2
8489,PBNCategory_77,"The risks of unauthorized access, misuse, corr...",Risk_3241,Unauthorized access to personal data,Risk,Risk
13606,PBNCategory_151,Digital tools and technologies facilitating va...,Technology_3724,Appliances,Technology,Technology
4263,PBNCategory_63,The challenges of urbanization and population ...,Risk_1446,Streets re-designed,Risk,Risk


In [10]:
print( df.TYPE2.unique(), "\n",df.TYPE.unique())

['Benef' 'Stakeholder' 'Risk' 'Mitigation' 'Technology' 'Theme'
 'Intervention' 'Enabler'] 
 ['Benef' 'Stakeholder' 'Risk' 'Mitigation' 'Technology' 'BP_Theme'
 'BP_Intervention' 'BP_Enabler']


In [11]:
INSTRUCT = "You are an international expert on contagious diseases spread in infrastructure, in particular green buildings neighbourhoods. \
In this context, you are given a list of SSSs. Summarize in a few sentences, in a single paragraph, up to 500 words, what these SSSs represents. \
This summary should allow any reader to determine if the individuals in the list belong to the group described in the summary. \
This description should be self-explanatory, and must not be a list of bullet points. The summary must not contain 'this list', This list..' or 'The list..'. \
The text should not contain 'The SSSs in this list' but rather 'This category of SSSs describes ..'. \
"

In [14]:
KEYS = {}
print(len(df.Category.unique()))
for uCategory in df.Category.unique():

    DF = df[df.Category == uCategory]
    PATH = "llm_cache/"+uCategory
    LST = "* "+"\n* ".join(DF.Label) 
    if not os.path.exists(PATH):
        WHAT = INSTRUCT.replace("SSS",DF.TYPE2.iloc[0]).replace("Benefs","Benenefits")
        #print(WHAT)
        summary = ask_GPT3(WHAT, LST)
        svt(PATH,summary)
        #print(uCategory,"\t---\t",summary)
    else:
        summary = ldt(PATH)
   
    while "\n\n" in summary:
        # while (len(summary) > 250) or (len(summary) < 20) or  summary.count(',') > 5 or summary.startswith("The list") or summary.startswith("This list")  :
        print("!!! Still too long or too short\n\n"+summary)
        summary = ask_GPT3(WHAT, LST)
        svt(PATH,summary)

    KEYS[uCategory] = summary


330
NOW: 11/01/2023, 14:38:44 --> 11/01/2023, 14:38:49
NOW: 11/01/2023, 14:38:49 --> 11/01/2023, 14:38:54
NOW: 11/01/2023, 14:38:54 --> 11/01/2023, 14:38:59
NOW: 11/01/2023, 14:38:59 --> 11/01/2023, 14:39:09
NOW: 11/01/2023, 14:39:09 --> 11/01/2023, 14:39:16
NOW: 11/01/2023, 14:39:16 --> 11/01/2023, 14:39:23
NOW: 11/01/2023, 14:39:23 --> 11/01/2023, 14:39:27
NOW: 11/01/2023, 14:39:27 --> 11/01/2023, 14:39:30
NOW: 11/01/2023, 14:39:30 --> 11/01/2023, 14:39:34
NOW: 11/01/2023, 14:39:34 --> 11/01/2023, 14:39:37
NOW: 11/01/2023, 14:39:37 --> 11/01/2023, 14:39:39
NOW: 11/01/2023, 14:39:39 --> 11/01/2023, 14:39:43
NOW: 11/01/2023, 14:39:43 --> 11/01/2023, 14:39:46
NOW: 11/01/2023, 14:39:46 --> 11/01/2023, 14:39:50
NOW: 11/01/2023, 14:39:50 --> 11/01/2023, 14:39:52
NOW: 11/01/2023, 14:39:52 --> 11/01/2023, 14:39:55
NOW: 11/01/2023, 14:39:55 --> 11/01/2023, 14:39:58
NOW: 11/01/2023, 14:39:58 --> 11/01/2023, 14:39:59
NOW: 11/01/2023, 14:39:59 --> 11/01/2023, 14:40:03
NOW: 11/01/2023, 14:40:03 -

In [194]:
CATS = pd.DataFrame.from_dict(KEYS,orient='index' ).reset_index()
CATS.columns = ["category","CatDescr"]
print(len(CATS))
CATS.to_parquet("outputs/CATS.parquet.gzip",compression="gzip")
CATS.head(3)

330


,category,CatDescr
0,PBNCategory_0,This category of Benenefits describes the vari...
1,PBNCategory_239,This category of Stakeholders describes indivi...
2,PBNCategory_50,This category of Risks describes various aspec...


In [16]:
dfRisks = dfRisk.merge(CATS,on="category",how="left")
dfRisks

,Risk,mitig,subject,owner,category,CatDescr
0,Risk_0,Mitigation_0,NaN,NaN,PBNCategory_50,This category of Risks describes various aspec...
1,Risk_828,Mitigation_416,NaN,NaN,PBNCategory_50,This category of Risks describes various aspec...
2,Risk_459,Mitigation_649,NaN,NaN,PBNCategory_50,This category of Risks describes various aspec...
3,Risk_642,Mitigation_988,Stakeholder_68,Stakeholder_394,PBNCategory_50,This category of Risks describes various aspec...
4,Risk_642,Mitigation_988,Stakeholder_68,Stakeholder_46,PBNCategory_50,This category of Risks describes various aspec...
...,...,...,...,...,...,...
40527,Risk_3370,Mitigation_5173,NaN,NaN,PBNCategory_50,This category of Risks describes various aspec...
40528,Risk_3370,Mitigation_5174,NaN,NaN,PBNCategory_50,This category of Risks describes various aspec...
40529,Risk_3392,Mitigation_5219,NaN,NaN,PBNCategory_66,This category of Risks describes various issue...
40530,Risk_3396,Mitigation_5224,NaN,NaN,PBNCategory_98,This category of Risks represents a wide range...


In [17]:
SUBJ = dfRisks.groupby(["category","subject"]).count().CatDescr.reset_index().sort_values("CatDescr",ascending=False)
print(len(SUBJ.category.unique()))

subjs  = {}
for CAT in SUBJ.category.unique():
    DF = SUBJ[SUBJ.category == CAT]
    subjs[CAT] = list(DF[:5].subject)

OWN = dfRisks.groupby(["category","owner"]).count().CatDescr.reset_index().sort_values("CatDescr",ascending=False)
print(len(OWN.category.unique()))

owners  = {}
for CAT in OWN.category.unique():
    DF = OWN[OWN.category == CAT]
    owners[CAT] = list(DF[:5].owner)

MITIG = dfRisks.groupby(["category","mitig"]).count().CatDescr.reset_index().sort_values("CatDescr",ascending=False)
print(len(MITIG.category.unique()))

Mitigs  = {}
for CAT in MITIG.category.unique():
    DF = MITIG[MITIG.category == CAT]
    Mitigs[CAT] = list(DF[:5].mitig)

Techs = {}
for CAT in MITIG.category.unique():
    L = list(set(list(dfRisks[dfRisks.category == CAT].mitig)))
    DFM = dfMitig[dfMitig.mitig.isin(L)]
    MM = DFM.groupby(["tech"]).count().mitig.reset_index().sort_values("mitig",ascending=False)
    Techs[CAT] = list(MM[:10].tech)
    
Benefs = {}
for CAT in MITIG.category.unique():
    L = list(set(list(dfRisks[dfRisks.category == CAT].mitig)))
    DFM = dfMitig[dfMitig.mitig.isin(L)]
    x = DFM.groupby(["benefit"]).count().mitig.reset_index().sort_values("mitig",ascending=False)
    Benefs[CAT] = list(x[:3].benefit)

50
50
50


In [18]:
dfRisks

,Risk,mitig,subject,owner,category,CatDescr
0,Risk_0,Mitigation_0,NaN,NaN,PBNCategory_50,This category of Risks describes various aspec...
1,Risk_828,Mitigation_416,NaN,NaN,PBNCategory_50,This category of Risks describes various aspec...
2,Risk_459,Mitigation_649,NaN,NaN,PBNCategory_50,This category of Risks describes various aspec...
3,Risk_642,Mitigation_988,Stakeholder_68,Stakeholder_394,PBNCategory_50,This category of Risks describes various aspec...
4,Risk_642,Mitigation_988,Stakeholder_68,Stakeholder_46,PBNCategory_50,This category of Risks describes various aspec...
...,...,...,...,...,...,...
40527,Risk_3370,Mitigation_5173,NaN,NaN,PBNCategory_50,This category of Risks describes various aspec...
40528,Risk_3370,Mitigation_5174,NaN,NaN,PBNCategory_50,This category of Risks describes various aspec...
40529,Risk_3392,Mitigation_5219,NaN,NaN,PBNCategory_66,This category of Risks describes various issue...
40530,Risk_3396,Mitigation_5224,NaN,NaN,PBNCategory_98,This category of Risks represents a wide range...


In [19]:
INSTRUCT = "You are an international expert on contagious diseases spread in infrastructure, in particular green buildings neighbourhoods. \
In this context, you are given a description of 'Blueprint' that is a mitigation against a type of risks, and which includes risk descriptions, stakeholders, technologies, etc.\
Considering the whole text you are given, write four paragraphs of up to 200 words each. The first paragraph introduces the context of this blueprint, followed by a second paragraph that provides an overarching summary of the risks that need to be tackled with this blueprint.\
The third paragraph summarizes the mitigation measures that can be deployed. The fourth paragraph summarizes in 100 words the benefits of implementing this mitigation measure (or blueprint).\
Each paragraph must start with '* '."

In [36]:
dfRisks[~dfRisks.category.isna()].category.unique()

array(['PBNCategory_50', 'PBNCategory_97', 'PBNCategory_51',
       'PBNCategory_52', 'PBNCategory_53', 'PBNCategory_83',
       'PBNCategory_54', 'PBNCategory_55', 'PBNCategory_56',
       'PBNCategory_95', 'PBNCategory_65', 'PBNCategory_75',
       'PBNCategory_78', 'PBNCategory_63', 'PBNCategory_57',
       'PBNCategory_85', 'PBNCategory_64', 'PBNCategory_58',
       'PBNCategory_59', 'PBNCategory_81', 'PBNCategory_62',
       'PBNCategory_80', 'PBNCategory_60', 'PBNCategory_61',
       'PBNCategory_76', 'PBNCategory_72', 'PBNCategory_88',
       'PBNCategory_89', 'PBNCategory_71', 'PBNCategory_66',
       'PBNCategory_67', 'PBNCategory_68', 'PBNCategory_69',
       'PBNCategory_70', 'PBNCategory_74', 'PBNCategory_73',
       'PBNCategory_77', 'PBNCategory_84', 'PBNCategory_91',
       'PBNCategory_79', 'PBNCategory_87', 'PBNCategory_82',
       'PBNCategory_98', 'PBNCategory_93', 'PBNCategory_86',
       'PBNCategory_90', 'PBNCategory_96', 'PBNCategory_92',
       'PBNCategory_94',

In [73]:
BPPS = {}
for CAT in dfRisks[~dfRisks.category.isna()].category.unique():
    print(CAT)
    BPPS[CAT] = {}
    #print(CAT,"\t",eval("onto.PBN__"+CAT+".label[0]"))
    PP = "# Blueprint data\n\n"
    PP += "## Description of the risk\n\n"+KEYS[CAT]+"\n\n"
    PP += "\n## People impacted by the risk\n\n"
    for k in subjs[CAT]:
        A = eval("onto.PBN__"+k)
        A = A.label[0]
        PP+= "* "+A+"\n"
    PP += "\n## Mitigations measures of the blueprint\n\n"
    for k in Mitigs[CAT]:
        A = eval("onto.PBN__"+k)
        A = A.label[0]
        PP+= "* "+A+"\n"
    PP += "\n## Stakeholders who could implement the mitigation measures\n\n"
    for k in owners[CAT]:
        A = eval("onto.PBN__"+k)
        A = A.label[0]
        PP+= "* "+A+"\n"
    PP += "\n## Technologies of the blueprint\n\n"
    for k in Techs[CAT]:
        A = eval("onto.PBN__"+k)
        A = A.label[0]
        PP+= "* "+A+"\n"
    PP += "\n## Benefits of the blueprint\n\n"
    for k in Benefs[CAT]:
        A = eval("onto.PBN__"+k)
        A = A.label[0]
        PP+= "* "+A+"\n"
        
    PATH = "llm_cache/Blueprint_"+CAT
    if not os.path.exists(PATH):
        WHAT = INSTRUCT
        #print(WHAT)
        summary = ask_GPT4(WHAT, PP)
        svt(PATH,summary)
        print(CAT,"\t---\t",summary)
    else:
        summary = ldt(PATH)
    BPPS[CAT]["summary"] = summary

PBNCategory_50
PBNCategory_97
PBNCategory_51
PBNCategory_52
PBNCategory_53
PBNCategory_83
PBNCategory_54
PBNCategory_55
PBNCategory_56
PBNCategory_95
PBNCategory_65
PBNCategory_75
PBNCategory_78
PBNCategory_63
PBNCategory_57
PBNCategory_85
PBNCategory_64
PBNCategory_58
PBNCategory_59
PBNCategory_81
PBNCategory_62
PBNCategory_80
PBNCategory_60
PBNCategory_61
PBNCategory_76
PBNCategory_72
PBNCategory_88
PBNCategory_89
PBNCategory_71
PBNCategory_66
PBNCategory_67
PBNCategory_68
PBNCategory_69
PBNCategory_70
PBNCategory_74
PBNCategory_73
PBNCategory_77
PBNCategory_84
PBNCategory_91
PBNCategory_79
PBNCategory_87
PBNCategory_82
PBNCategory_98
PBNCategory_93
PBNCategory_86
PBNCategory_90
PBNCategory_96
PBNCategory_92
PBNCategory_94
PBNCategory_99


In [74]:
INSTRUCTDetails = "You are an international expert on contagious diseases spread in infrastructure, in particular green buildings neighbourhoods. \
In this context, you are given a description of 'Blueprint' that is a mitigation against a type of risks, and which includes risk descriptions, stakeholders, technologies, etc.\
Write 4 paragraphs of up to 200 words each. The first is a recommendation to implement the blueprint in practice, covering a clear and simple description of the blueprint implementation, in simple terms.\
The second is a single sentence that states who can implement the blueprint.\
The third paragraph is a single word, either 'Building' or 'Neighbourhood', depending on the scale of the impact of the blueprint.\
The fourth paragraph provides a very simple summary of the risks listed in the blueprint.\
Each paragraph must start with '* '."

In [75]:
dfRisks[~dfRisks.category.isna()].category.unique()

array(['PBNCategory_50', 'PBNCategory_97', 'PBNCategory_51',
       'PBNCategory_52', 'PBNCategory_53', 'PBNCategory_83',
       'PBNCategory_54', 'PBNCategory_55', 'PBNCategory_56',
       'PBNCategory_95', 'PBNCategory_65', 'PBNCategory_75',
       'PBNCategory_78', 'PBNCategory_63', 'PBNCategory_57',
       'PBNCategory_85', 'PBNCategory_64', 'PBNCategory_58',
       'PBNCategory_59', 'PBNCategory_81', 'PBNCategory_62',
       'PBNCategory_80', 'PBNCategory_60', 'PBNCategory_61',
       'PBNCategory_76', 'PBNCategory_72', 'PBNCategory_88',
       'PBNCategory_89', 'PBNCategory_71', 'PBNCategory_66',
       'PBNCategory_67', 'PBNCategory_68', 'PBNCategory_69',
       'PBNCategory_70', 'PBNCategory_74', 'PBNCategory_73',
       'PBNCategory_77', 'PBNCategory_84', 'PBNCategory_91',
       'PBNCategory_79', 'PBNCategory_87', 'PBNCategory_82',
       'PBNCategory_98', 'PBNCategory_93', 'PBNCategory_86',
       'PBNCategory_90', 'PBNCategory_96', 'PBNCategory_92',
       'PBNCategory_94',

In [76]:
for CAT in dfRisks[~dfRisks.category.isna()].category.unique():
    print(CAT)
    #print(CAT,"\t",eval("onto.PBN__"+CAT+".label[0]"))
    PP = "# Blueprint data\n\n"
    PP += "## Description of the risk\n\n"+KEYS[CAT]+"\n\n"
    PP += "\n## People impacted by the risk\n\n"
    for k in subjs[CAT]:
        A = eval("onto.PBN__"+k)
        A = A.label[0]
        PP+= "* "+A+"\n"
    PP += "\n## Mitigations measures of the blueprint\n\n"
    for k in Mitigs[CAT]:
        A = eval("onto.PBN__"+k)
        A = A.label[0]
        PP+= "* "+A+"\n"
    PP += "\n## Stakeholders who could implement the mitigation measures\n\n"
    for k in owners[CAT]:
        A = eval("onto.PBN__"+k)
        A = A.label[0]
        PP+= "* "+A+"\n"
    PP += "\n## Technologies of the blueprint\n\n"
    for k in Techs[CAT]:
        A = eval("onto.PBN__"+k)
        A = A.label[0]
        PP+= "* "+A+"\n"
    PP += "\n## Benefits of the blueprint\n\n"
    for k in Benefs[CAT]:
        A = eval("onto.PBN__"+k)
        A = A.label[0]
        PP+= "* "+A+"\n"
    PP += "\n\n## Proposition of the blueprint\n\n"+BPPS[CAT]["summary"]
    
    PATH = "llm_cache/BlueprintDetails_"+CAT
    if not os.path.exists(PATH):
        WHAT = INSTRUCTDetails
        #print(WHAT)
        summary = ask_GPT4(WHAT, PP)
        svt(PATH,summary)
        print(CAT,"\t---\t",summary)
    else:
        summary = ldt(PATH)
    BPPS[CAT]["detail"] = summary

PBNCategory_50
PBNCategory_97
PBNCategory_51
PBNCategory_52
PBNCategory_53
PBNCategory_83
PBNCategory_54
PBNCategory_55
PBNCategory_56
PBNCategory_95
PBNCategory_65
PBNCategory_75
PBNCategory_78
PBNCategory_63
PBNCategory_57
PBNCategory_85
PBNCategory_64
PBNCategory_58
PBNCategory_59
PBNCategory_81
PBNCategory_62
PBNCategory_80
PBNCategory_60
PBNCategory_61
PBNCategory_76
PBNCategory_72
PBNCategory_88
PBNCategory_89
PBNCategory_71
PBNCategory_66
PBNCategory_67
PBNCategory_68
PBNCategory_69
PBNCategory_70
PBNCategory_74
PBNCategory_73
PBNCategory_77
PBNCategory_84
PBNCategory_91
PBNCategory_79
PBNCategory_87
PBNCategory_82
PBNCategory_98
PBNCategory_93
PBNCategory_86
PBNCategory_90
PBNCategory_96
PBNCategory_92
PBNCategory_94
PBNCategory_99


In [117]:
INSTRUCTDetailsTitle = "You are an international expert on contagious diseases spread in infrastructure, in particular green buildings neighbourhoods. \
In this context, you are given a description of 'Blueprint' that is a mitigation against a type of risks, and which includes risk descriptions, stakeholders, technologies, etc.\
Give the title (in up to 20 words). The title must describe an action to tackle the problem and as such must start with the verb 'Implementing ..'.\
The word 'title', blueprint', the expression 'Green Building Neighbourhoods', the expression 'green building' or the expression 'Green Building Neighborhoods' absolutely must not appear in the title.\
The title cannot be generic, for example 'Mitigating Risks in Green Building Communities' - it must be specific to the topic at hand and contain the main risks."

In [121]:
D = dfRisks[~dfRisks.category.isna()]
for CAT in D.sample(len(D)).category.unique():
    print(CAT)
    #print(CAT,"\t",eval("onto.PBN__"+CAT+".label[0]"))
    PP = "# Description of the blueprint\n\n"
    #PP += "## Description of the risk\n\n"+KEYS[CAT]+"\n\n"
    PP += "\n## Mitigations measures of the blueprint\n\n"
    for k in Mitigs[CAT]:
        A = eval("onto.PBN__"+k)
        A = A.label[0]
        PP+= "* "+A+"\n"

    PATH = "llm_cache/BlueprintTitle_"+CAT
    
    if not os.path.exists(PATH):
        WHAT = INSTRUCTDetailsTitle
        #print(WHAT)
        summary = ask_GPT4(WHAT, PP)
        svt(PATH,summary)
        print(uCategory,"\t---\t",summary)
    else:
        summary = ldt(PATH)
    summary = summary.replace("Title:","").strip().replace('"',"")
    while (not summary.startswith("Implementing")) or  ('Title' in summary) or ('Blueprint' in summary) or ('Blueprint for' in summary) or ("Neighbor" in summary) or ("Green Building" in summary):
        summary = ask_GPT4(WHAT, PP)
        summary = summary.replace("Title:","").strip().replace('"',"")
        svt(PATH,summary)
        #print(summary)
    print(CAT,summary)
    BPPS[CAT]["title"] = summary

PBNCategory_52
PBNCategory_52 Implementing Mental Health Mitigation Strategies in Eco-Friendly Residential Communities
PBNCategory_63
PBNCategory_63 Implementing Occupancy Controls and Physical Distancing in Sustainable Architecture to Mitigate Contagious Disease Spread
PBNCategory_95
PBNCategory_95 Implementing Filtration and Protective Measures to Minimize Contagious Diseases in Sustainable Infrastructure
PBNCategory_89
PBNCategory_89 Implementing Disease Containment Approaches in Sustainable Infrastructure Through Occupancy Limits and Promotion of Outdoor Spaces
PBNCategory_57
PBNCategory_57 Implementing Disease Containment Strategies in Environmentally Sustained Residential Areas
PBNCategory_67
PBNCategory_67 Implementing Contagious Disease Control and Privacy Protection in Sustainable Infrastructure Communities
PBNCategory_91
PBNCategory_91 Implementing Disease Containment Strategies and Technological Solutions in Sustainable Infrastructure
PBNCategory_93
PBNCategory_93 Implementi

In [137]:
for CAT in BPPS.keys():
    A = ": ".join(BPPS[CAT]["title"].split(" in "))
    A = [x.strip() for x in A.split(":")]
    if len(A) == 2:
        C = ": ".join(A[:-1])
        print(len(A),C)
    elif len(A) == 3:
        C = A[0]+": "+A[2]
        print(C)
    elif len(A) == 1:
        C =A[0]
        print(C)
    else:
        C = A
        print(len(A),A)
    BPPS[CAT]["title_clean"] = C

2 Implementing Regular Cleaning, WBE and Quarantine Measures for Infectious Diseases Mitigation
2 Implementing Wastewater Monitoring and Enhanced Sanitation Measures to Mitigate SARS-CoV-2 Spread
2 Implementing Effective Measures to Combat Contagious Diseases
2 Implementing Mental Health Mitigation Strategies
2 Implementing Collaborative Measures to Combat Contagious Diseases
2 Implementing Early-Warning Systems and Safety Protocols Against COVID-19 Spread
2 Implementing Contagious Disease Control through Social Distancing and Digital Engagement
2 Implementing Digital Tracing and Proactive Infection Control Measures
Implementing Disease Containment Strategies: Managing Contagious Risks with Social Distancing and Enhanced Sanitation Measures
2 Implementing Filtration and Protective Measures to Minimize Contagious Diseases
Implementing Contagious Disease Control: Stakeholders, Strategies, and Technologies
2 Implementing Vaccination Strategies and Awareness Campaigns
2 Implementing Contag

# Consolidation

In [186]:
ALLCAT = []
for CAT in dfRisks.category.unique():
    if not str(CAT) == "nan":
        #print(CAT)
        #print(CAT,"\t",eval("onto.PBN__"+CAT+".label[0]"))
        SH_Subject = subjs[CAT]
        SH_Owner = owners[CAT]    
        Mitigations = Mitigs[CAT]
        Technology = Techs[CAT]
        Benefits = Benefs[CAT]
        RISKS = KEYS[CAT]
        ABP_Summary = BPPS[CAT]["summary"]
        ABP_Details = BPPS[CAT]["detail"]
        ABP_Title = BPPS[CAT]["title_clean"]
        ALLCAT.append([CAT,ABP_Title,RISKS,SH_Subject,SH_Owner,Mitigations,Technology,Benefits,ABP_Summary,ABP_Details])
DFFinal = pd.DataFrame(ALLCAT)
DFFinal.columns = ["category","title","risks","subject","owner","mitig","techs","benefs","summary","details"]
print(len(DFFinal)) 
DFFinal

50


,category,title,risks,subject,owner,mitig,techs,benefs,summary,details
0,PBNCategory_50,"Implementing Regular Cleaning, WBE and Quarant...",This category of Risks describes various aspec...,"[Stakeholder_68, Stakeholder_9, Stakeholder_31...","[Stakeholder_269, Stakeholder_46, Stakeholder_...","[Mitigation_1587, Mitigation_988, Mitigation_1...","[Technology_22, Technology_358, Technology_185...","[Benef_5, Benef_1206, Benef_988]",* The Blueprint outlined addresses the pressin...,"* I recommend implementing the Blueprint, whic..."
1,PBNCategory_97,Implementing Wastewater Monitoring and Enhance...,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_2, Stakeholder_104...","[Stakeholder_1046, Stakeholder_413, Stakeholde...","[Mitigation_2162, Mitigation_2164, Mitigation_...","[Technology_3066, Technology_3067, Technology_...","[Benef_988, Benef_1206, Benef_5]",The context of this Blueprint revolves around ...,"* The ""Blueprint"" provides a comprehensive gui..."
2,PBNCategory_51,Implementing Effective Measures to Combat Cont...,This category of Risks describes a range of is...,"[Stakeholder_7, Stakeholder_562, Stakeholder_9...","[Stakeholder_627, Stakeholder_73, Stakeholder_...","[Mitigation_1853, Mitigation_2467, Mitigation_...","[Technology_22, Technology_23, Technology_2579...","[Benef_1366, Benef_632, Benef_631]",* The context of this blueprint derived from t...,* I strongly recommend implementing the 'Bluep...
3,PBNCategory_52,Implementing Mental Health Mitigation Strategies,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_68, Stakeholder_29...","[Stakeholder_121, Stakeholder_47, Stakeholder_...","[Mitigation_2128, Mitigation_2315, Mitigation_...","[Technology_22, Technology_146, Technology_280...","[Benef_436, Benef_153, Benef_11]",* The provided blueprint revolves around menta...,"* Based on the analysis, I would highly recomm..."
4,PBNCategory_53,Implementing Collaborative Measures to Combat ...,This category of Risks describes a range of is...,"[Stakeholder_9, Stakeholder_222, Stakeholder_1...","[Stakeholder_73, Stakeholder_1357, Stakeholder...","[Mitigation_2674, Mitigation_2230, Mitigation_...","[Technology_22, Technology_3066, Technology_31...","[Benef_860, Benef_166, Benef_1022]",* The blueprint is a crucial mitigation strate...,* The 'Blueprint' is essentially a thorough ac...
5,PBNCategory_83,Implementing Early-Warning Systems and Safety ...,This category of Risks describes a wide range ...,"[Stakeholder_72, Stakeholder_9, Stakeholder_47...","[Stakeholder_196, Stakeholder_47, Stakeholder_...","[Mitigation_1796, Mitigation_100, Mitigation_2...","[Technology_22, Technology_2969, Technology_34...","[Benef_958, Benef_1194, Benef_549]",* In the context of green building neighbourho...,* The Blueprint recommendation focuses on the ...
6,PBNCategory_54,Implementing Contagious Disease Control throug...,This category of Risks describes various aspec...,"[Stakeholder_197, Stakeholder_9, Stakeholder_2...","[Stakeholder_216, Stakeholder_73, Stakeholder_...","[Mitigation_1652, Mitigation_2651, Mitigation_...","[Technology_22, Technology_358, Technology_248...","[Benef_232, Benef_368, Benef_1366]",* The context surrounding this blueprint focus...,* My professional recommendation is to fully i...
7,PBNCategory_55,Implementing Digital Tracing and Proactive Inf...,This category of Risks describes various aspec...,"[Stakeholder_9, Stakeholder_68, Stakeholder_59...","[Stakeholder_47, Stakeholder_73, Stakeholder_2...","[Mitigation_3154, Mitigation_3156, Mitigation_...","[Technology_22, Technology_341, Technology_23,...","[Benef_631, Benef_29, Benef_582]",* The 'Blueprint' presents a holistic approach...,* I would highly recommend implementing the 'B...
8,PBNCategory_56,Implementing Disease Containment Strategies: M...,This category of Risks describes various aspec...,"[Stakeholder_9, Stakeholder_68, Stakeholder_12...","[Stakeholder_20, Stakeholder_269, Stakeholder_...","[Mitigatio

In [187]:
DFFinal["details_list"] = DFFinal.details.apply(lambda X: [ x.replace("*","").strip() for x in X.split("\n") if len(x)>2])
DFFinal["details_list"]  = DFFinal["details_list"].apply(lambda X: "---".join(X))
DFFinal

,category,title,risks,subject,owner,mitig,techs,benefs,summary,details,details_list
0,PBNCategory_50,"Implementing Regular Cleaning, WBE and Quarant...",This category of Risks describes various aspec...,"[Stakeholder_68, Stakeholder_9, Stakeholder_31...","[Stakeholder_269, Stakeholder_46, Stakeholder_...","[Mitigation_1587, Mitigation_988, Mitigation_1...","[Technology_22, Technology_358, Technology_185...","[Benef_5, Benef_1206, Benef_988]",* The Blueprint outlined addresses the pressin...,"* I recommend implementing the Blueprint, whic...","I recommend implementing the Blueprint, which ..."
1,PBNCategory_97,Implementing Wastewater Monitoring and Enhance...,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_2, Stakeholder_104...","[Stakeholder_1046, Stakeholder_413, Stakeholde...","[Mitigation_2162, Mitigation_2164, Mitigation_...","[Technology_3066, Technology_3067, Technology_...","[Benef_988, Benef_1206, Benef_5]",The context of this Blueprint revolves around ...,"* The ""Blueprint"" provides a comprehensive gui...","The ""Blueprint"" provides a comprehensive guide..."
2,PBNCategory_51,Implementing Effective Measures to Combat Cont...,This category of Risks describes a range of is...,"[Stakeholder_7, Stakeholder_562, Stakeholder_9...","[Stakeholder_627, Stakeholder_73, Stakeholder_...","[Mitigation_1853, Mitigation_2467, Mitigation_...","[Technology_22, Technology_23, Technology_2579...","[Benef_1366, Benef_632, Benef_631]",* The context of this blueprint derived from t...,* I strongly recommend implementing the 'Bluep...,I strongly recommend implementing the 'Bluepri...
3,PBNCategory_52,Implementing Mental Health Mitigation Strategies,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_68, Stakeholder_29...","[Stakeholder_121, Stakeholder_47, Stakeholder_...","[Mitigation_2128, Mitigation_2315, Mitigation_...","[Technology_22, Technology_146, Technology_280...","[Benef_436, Benef_153, Benef_11]",* The provided blueprint revolves around menta...,"* Based on the analysis, I would highly recomm...","Based on the analysis, I would highly recommen..."
4,PBNCategory_53,Implementing Collaborative Measures to Combat ...,This category of Risks describes a range of is...,"[Stakeholder_9, Stakeholder_222, Stakeholder_1...","[Stakeholder_73, Stakeholder_1357, Stakeholder...","[Mitigation_2674, Mitigation_2230, Mitigation_...","[Technology_22, Technology_3066, Technology_31...","[Benef_860, Benef_166, Benef_1022]",* The blueprint is a crucial mitigation strate...,* The 'Blueprint' is essentially a thorough ac...,The 'Blueprint' is essentially a thorough acti...
5,PBNCategory_83,Implementing Early-Warning Systems and Safety ...,This category of Risks describes a wide range ...,"[Stakeholder_72, Stakeholder_9, Stakeholder_47...","[Stakeholder_196, Stakeholder_47, Stakeholder_...","[Mitigation_1796, Mitigation_100, Mitigation_2...","[Technology_22, Technology_2969, Technology_34...","[Benef_958, Benef_1194, Benef_549]",* In the context of green building neighbourho...,* The Blueprint recommendation focuses on the ...,The Blueprint recommendation focuses on the pr...
6,PBNCategory_54,Implementing Contagious Disease Control throug...,This category of Risks describes various aspec...,"[Stakeholder_197, Stakeholder_9, Stakeholder_2...","[Stakeholder_216, Stakeholder_73, Stakeholder_...","[Mitigation_1652, Mitigation_2651, Mitigation_...","[Technology_22, Technology_358, Technology_248...","[Benef_232, Benef_368, Benef_1366]",* The context surrounding this blueprint focus...,* My professional recommendation is to fully i...,My professional recommendation is to fully imp...
7,PBNCategory_55,Implementing Digital Tracing and Proactive Inf...,This category of Risks describes various aspec...,"[Stakeholder_9, Stakeholder_68, Stakeholder_59...","[Stakeholder_47, Stakeholder_73, Stakeholder_2...","[Mitigation_3154, Mitigation_3156, Mitigation_...","[Technology_22, Technology_341, Technology_23,.

In [188]:
DFFinal["summary_list"] = DFFinal.summary.apply(lambda X: [ x.replace("*","").strip() for x in X.split("\n") if len(x)>2])
DFFinal["summary_list"]  = DFFinal["summary_list"].apply(lambda X: "---".join(X))
DFFinal

,category,title,risks,subject,owner,mitig,techs,benefs,summary,details,details_list,summary_list
0,PBNCategory_50,"Implementing Regular Cleaning, WBE and Quarant...",This category of Risks describes various aspec...,"[Stakeholder_68, Stakeholder_9, Stakeholder_31...","[Stakeholder_269, Stakeholder_46, Stakeholder_...","[Mitigation_1587, Mitigation_988, Mitigation_1...","[Technology_22, Technology_358, Technology_185...","[Benef_5, Benef_1206, Benef_988]",* The Blueprint outlined addresses the pressin...,"* I recommend implementing the Blueprint, whic...","I recommend implementing the Blueprint, which ...",The Blueprint outlined addresses the pressing ...
1,PBNCategory_97,Implementing Wastewater Monitoring and Enhance...,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_2, Stakeholder_104...","[Stakeholder_1046, Stakeholder_413, Stakeholde...","[Mitigation_2162, Mitigation_2164, Mitigation_...","[Technology_3066, Technology_3067, Technology_...","[Benef_988, Benef_1206, Benef_5]",The context of this Blueprint revolves around ...,"* The ""Blueprint"" provides a comprehensive gui...","The ""Blueprint"" provides a comprehensive guide...",The context of this Blueprint revolves around ...
2,PBNCategory_51,Implementing Effective Measures to Combat Cont...,This category of Risks describes a range of is...,"[Stakeholder_7, Stakeholder_562, Stakeholder_9...","[Stakeholder_627, Stakeholder_73, Stakeholder_...","[Mitigation_1853, Mitigation_2467, Mitigation_...","[Technology_22, Technology_23, Technology_2579...","[Benef_1366, Benef_632, Benef_631]",* The context of this blueprint derived from t...,* I strongly recommend implementing the 'Bluep...,I strongly recommend implementing the 'Bluepri...,The context of this blueprint derived from the...
3,PBNCategory_52,Implementing Mental Health Mitigation Strategies,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_68, Stakeholder_29...","[Stakeholder_121, Stakeholder_47, Stakeholder_...","[Mitigation_2128, Mitigation_2315, Mitigation_...","[Technology_22, Technology_146, Technology_280...","[Benef_436, Benef_153, Benef_11]",* The provided blueprint revolves around menta...,"* Based on the analysis, I would highly recomm...","Based on the analysis, I would highly recommen...",The provided blueprint revolves around mental ...
4,PBNCategory_53,Implementing Collaborative Measures to Combat ...,This category of Risks describes a range of is...,"[Stakeholder_9, Stakeholder_222, Stakeholder_1...","[Stakeholder_73, Stakeholder_1357, Stakeholder...","[Mitigation_2674, Mitigation_2230, Mitigation_...","[Technology_22, Technology_3066, Technology_31...","[Benef_860, Benef_166, Benef_1022]",* The blueprint is a crucial mitigation strate...,* The 'Blueprint' is essentially a thorough ac...,The 'Blueprint' is essentially a thorough acti...,The blueprint is a crucial mitigation strategy...
5,PBNCategory_83,Implementing Early-Warning Systems and Safety ...,This category of Risks describes a wide range ...,"[Stakeholder_72, Stakeholder_9, Stakeholder_47...","[Stakeholder_196, Stakeholder_47, Stakeholder_...","[Mitigation_1796, Mitigation_100, Mitigation_2...","[Technology_22, Technology_2969, Technology_34...","[Benef_958, Benef_1194, Benef_549]",* In the context of green building neighbourho...,* The Blueprint recommendation focuses on the ...,The Blueprint recommendation focuses on the pr...,In the context of green building neighbourhood...
6,PBNCategory_54,Implementing Contagious Disease Control throug...,This category of Risks describes various aspec...,"[Stakeholder_197, Stakeholder_9, Stakeholder_2...","[Stakeholder_216, Stakeholder_73, Stakeholder_...","[Mitigation_1652, Mitigation_2651, Mitigation_...","[Technology_22, Technology_358, Technology_248...","[Benef_232, Benef_368, Benef_1366]",* The context surrounding this blueprint focus...,* My professional recommendation is to fully i...,My professional recommendation is to fully imp...,The cont

In [189]:
DFFinal[["summ_context","summ_risk","summ_mitig","summ_benefits"]] = DFFinal["summary_list"].str.split('---', expand=True)
DFFinal

,category,title,risks,subject,owner,mitig,techs,benefs,summary,details,details_list,summary_list,summ_context,summ_risk,summ_mitig,summ_benefits
0,PBNCategory_50,"Implementing Regular Cleaning, WBE and Quarant...",This category of Risks describes various aspec...,"[Stakeholder_68, Stakeholder_9, Stakeholder_31...","[Stakeholder_269, Stakeholder_46, Stakeholder_...","[Mitigation_1587, Mitigation_988, Mitigation_1...","[Technology_22, Technology_358, Technology_185...","[Benef_5, Benef_1206, Benef_988]",* The Blueprint outlined addresses the pressin...,"* I recommend implementing the Blueprint, whic...","I recommend implementing the Blueprint, which ...",The Blueprint outlined addresses the pressing ...,The Blueprint outlined addresses the pressing ...,The risks associated with the prevailing COVID...,For the Blueprint to be effective in mitigatin...,The implementation of the Blueprint offers a p...
1,PBNCategory_97,Implementing Wastewater Monitoring and Enhance...,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_2, Stakeholder_104...","[Stakeholder_1046, Stakeholder_413, Stakeholde...","[Mitigation_2162, Mitigation_2164, Mitigation_...","[Technology_3066, Technology_3067, Technology_...","[Benef_988, Benef_1206, Benef_5]",The context of this Blueprint revolves around ...,"* The ""Blueprint"" provides a comprehensive gui...","The ""Blueprint"" provides a comprehensive guide...",The context of this Blueprint revolves around ...,The context of this Blueprint revolves around ...,The risks identified within this blueprint foc...,The mitigation measures laid out in this bluep...,Implementing the mitigation measures laid out ...
2,PBNCategory_51,Implementing Effective Measures to Combat Cont...,This category of Risks describes a range of is...,"[Stakeholder_7, Stakeholder_562, Stakeholder_9...","[Stakeholder_627, Stakeholder_73, Stakeholder_...","[Mitigation_1853, Mitigation_2467, Mitigation_...","[Technology_22, Technology_23, Technology_2579...","[Benef_1366, Benef_632, Benef_631]",* The context of this blueprint derived from t...,* I strongly recommend implementing the 'Bluep...,I strongly recommend implementing the 'Bluepri...,The context of this blueprint derived from the...,The context of this blueprint derived from the...,The listed risks comprise domestic and intimat...,The blueprint underlines a strategic approach ...,The implementation of this blueprint holds sig...
3,PBNCategory_52,Implementing Mental Health Mitigation Strategies,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_68, Stakeholder_29...","[Stakeholder_121, Stakeholder_47, Stakeholder_...","[Mitigation_2128, Mitigation_2315, Mitigation_...","[Technology_22, Technology_146, Technology_280...","[Benef_436, Benef_153, Benef_11]",* The provided blueprint revolves around menta...,"* Based on the analysis, I would highly recomm...","Based on the analysis, I would highly recommen...",The provided blueprint revolves around mental ...,The provided blueprint revolves around mental ...,Several risks are highlighted in the blueprint...,"To tackle these potential threats, a series of...",The implementation of this blueprint or these ...
4,PBNCategory_53,Implementing Collaborative Measures to Combat ...,This category of Risks describes a range of is...,"[Stakeholder_9, Stakeholder_222, Stakeholder_1...","[Stakeholder_73, Stakeholder_1357, Stakeholder...","[Mitigation_2674, Mitigation_2230, Mitigation_...","[Technology_22, Technology_3066, Technology_31...","[Benef_860, Benef_166, Benef_1022]",* The blueprint is a crucial mitigation strate...,* The 'Blueprint' is essentially a thorough ac...,The 'Blueprint' is essentially a thorough acti...,The blueprint is a crucial mitigation strategy...,The blueprint is a crucial mitigation strategy...,The focus of this blueprint deals with an exte...,The primary mitigation measures embedded withi...,Implementing these mitigation measures present...
5,PBNCategory_83,Implementing Ea

In [190]:
DFFinal[["det_recomm","det_who","det_scale","det_simple"]]  = DFFinal["details_list"].str.split('---', expand=True)
DFFinal

,category,title,risks,subject,owner,mitig,techs,benefs,summary,details,details_list,summary_list,summ_context,summ_risk,summ_mitig,summ_benefits,det_recomm,det_who,det_scale,det_simple
0,PBNCategory_50,"Implementing Regular Cleaning, WBE and Quarant...",This category of Risks describes various aspec...,"[Stakeholder_68, Stakeholder_9, Stakeholder_31...","[Stakeholder_269, Stakeholder_46, Stakeholder_...","[Mitigation_1587, Mitigation_988, Mitigation_1...","[Technology_22, Technology_358, Technology_185...","[Benef_5, Benef_1206, Benef_988]",* The Blueprint outlined addresses the pressin...,"* I recommend implementing the Blueprint, whic...","I recommend implementing the Blueprint, which ...",The Blueprint outlined addresses the pressing ...,The Blueprint outlined addresses the pressing ...,The risks associated with the prevailing COVID...,For the Blueprint to be effective in mitigatin...,The implementation of the Blueprint offers a p...,"I recommend implementing the Blueprint, which ...","Stakeholders such as healthcare institutions, ...",'Neighbourhood',The Blueprint lists numerous risks including t...
1,PBNCategory_97,Implementing Wastewater Monitoring and Enhance...,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_2, Stakeholder_104...","[Stakeholder_1046, Stakeholder_413, Stakeholde...","[Mitigation_2162, Mitigation_2164, Mitigation_...","[Technology_3066, Technology_3067, Technology_...","[Benef_988, Benef_1206, Benef_5]",The context of this Blueprint revolves around ...,"* The ""Blueprint"" provides a comprehensive gui...","The ""Blueprint"" provides a comprehensive guide...",The context of this Blueprint revolves around ...,The context of this Blueprint revolves around ...,The risks identified within this blueprint foc...,The mitigation measures laid out in this bluep...,Implementing the mitigation measures laid out ...,"The ""Blueprint"" provides a comprehensive guide...","Wastewater treatment plant operators, research...",Neighbourhood,The blueprint addresses several risks related ...
2,PBNCategory_51,Implementing Effective Measures to Combat Cont...,This category of Risks describes a range of is...,"[Stakeholder_7, Stakeholder_562, Stakeholder_9...","[Stakeholder_627, Stakeholder_73, Stakeholder_...","[Mitigation_1853, Mitigation_2467, Mitigation_...","[Technology_22, Technology_23, Technology_2579...","[Benef_1366, Benef_632, Benef_631]",* The context of this blueprint derived from t...,* I strongly recommend implementing the 'Bluep...,I strongly recommend implementing the 'Bluepri...,The context of this blueprint derived from the...,The context of this blueprint derived from the...,The listed risks comprise domestic and intimat...,The blueprint underlines a strategic approach ...,The implementation of this blueprint holds sig...,I strongly recommend implementing the 'Bluepri...,"Government entities, NGOs, and law enforcement...",'Neighbourhood',The risks detailed in 'Blueprint' primarily co...
3,PBNCategory_52,Implementing Mental Health Mitigation Strategies,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_68, Stakeholder_29...","[Stakeholder_121, Stakeholder_47, Stakeholder_...","[Mitigation_2128, Mitigation_2315, Mitigation_...","[Technology_22, Technology_146, Technology_280...","[Benef_436, Benef_153, Benef_11]",* The provided blueprint revolves around menta...,"* Based on the analysis, I would highly recomm...","Based on the analysis, I would highly recommen...",The provided blueprint revolves around mental ...,The provided blueprint revolves around mental ...,Several risks are highlighted in the blueprint...,"To tackle these potential threats, a series of...",The implementation of this blueprint or these ...,"Based on the analysis, I would highly recommen...","Healthcare providers, governments, mental heal...",'Neighbourhood',The risks targeted in this blueprint are menta...
4,PBNCategory_53,Implementing Collaborative Measures to Combat ...,This cate

In [191]:
DFFinal["SCALE"] = DFFinal.det_scale.apply(lambda x: "Neighbourhood" if "eighb" in x else "Building")
DFFinal

,category,title,risks,subject,owner,mitig,techs,benefs,summary,details,...,summary_list,summ_context,summ_risk,summ_mitig,summ_benefits,det_recomm,det_who,det_scale,det_simple,SCALE
0,PBNCategory_50,"Implementing Regular Cleaning, WBE and Quarant...",This category of Risks describes various aspec...,"[Stakeholder_68, Stakeholder_9, Stakeholder_31...","[Stakeholder_269, Stakeholder_46, Stakeholder_...","[Mitigation_1587, Mitigation_988, Mitigation_1...","[Technology_22, Technology_358, Technology_185...","[Benef_5, Benef_1206, Benef_988]",* The Blueprint outlined addresses the pressin...,"* I recommend implementing the Blueprint, whic...",...,The Blueprint outlined addresses the pressing ...,The Blueprint outlined addresses the pressing ...,The risks associated with the prevailing COVID...,For the Blueprint to be effective in mitigatin...,The implementation of the Blueprint offers a p...,"I recommend implementing the Blueprint, which ...","Stakeholders such as healthcare institutions, ...",'Neighbourhood',The Blueprint lists numerous risks including t...,Neighbourhood
1,PBNCategory_97,Implementing Wastewater Monitoring and Enhance...,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_2, Stakeholder_104...","[Stakeholder_1046, Stakeholder_413, Stakeholde...","[Mitigation_2162, Mitigation_2164, Mitigation_...","[Technology_3066, Technology_3067, Technology_...","[Benef_988, Benef_1206, Benef_5]",The context of this Blueprint revolves around ...,"* The ""Blueprint"" provides a comprehensive gui...",...,The context of this Blueprint revolves around ...,The context of this Blueprint revolves around ...,The risks identified within this blueprint foc...,The mitigation measures laid out in this bluep...,Implementing the mitigation measures laid out ...,"The ""Blueprint"" provides a comprehensive guide...","Wastewater treatment plant operators, research...",Neighbourhood,The blueprint addresses several risks related ...,Neighbourhood
2,PBNCategory_51,Implementing Effective Measures to Combat Cont...,This category of Risks describes a range of is...,"[Stakeholder_7, Stakeholder_562, Stakeholder_9...","[Stakeholder_627, Stakeholder_73, Stakeholder_...","[Mitigation_1853, Mitigation_2467, Mitigation_...","[Technology_22, Technology_23, Technology_2579...","[Benef_1366, Benef_632, Benef_631]",* The context of this blueprint derived from t...,* I strongly recommend implementing the 'Bluep...,...,The context of this blueprint derived from the...,The context of this blueprint derived from the...,The listed risks comprise domestic and intimat...,The blueprint underlines a strategic approach ...,The implementation of this blueprint holds sig...,I strongly recommend implementing the 'Bluepri...,"Government entities, NGOs, and law enforcement...",'Neighbourhood',The risks detailed in 'Blueprint' primarily co...,Neighbourhood
3,PBNCategory_52,Implementing Mental Health Mitigation Strategies,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_68, Stakeholder_29...","[Stakeholder_121, Stakeholder_47, Stakeholder_...","[Mitigation_2128, Mitigation_2315, Mitigation_...","[Technology_22, Technology_146, Technology_280...","[Benef_436, Benef_153, Benef_11]",* The provided blueprint revolves around menta...,"* Based on the analysis, I would highly recomm...",...,The provided blueprint revolves around mental ...,The provided blueprint revolves around mental ...,Several risks are highlighted in the blueprint...,"To tackle these potential threats, a series of...",The implementation of this blueprint or these ...,"Based on the analysis, I would highly recommen...","Healthcare providers, governments, mental heal...",'Neighbourhood',The risks targeted in this blueprint are menta...,Neighbourhood
4,PBNCategory_53,Implementing Collaborative Measures to Combat ...,This category of Risks describes a range of is...,"[Stakeholder_9, Stakeholder_222, Stakeholder_1...","[Stakeholder_73, Stakeholder_1357, Stakeholde

In [192]:
DFFinal.to_parquet("outputs/newBP.parquet.gzip",compression="gzip")
DFFinal

,category,title,risks,subject,owner,mitig,techs,benefs,summary,details,...,summary_list,summ_context,summ_risk,summ_mitig,summ_benefits,det_recomm,det_who,det_scale,det_simple,SCALE
0,PBNCategory_50,"Implementing Regular Cleaning, WBE and Quarant...",This category of Risks describes various aspec...,"[Stakeholder_68, Stakeholder_9, Stakeholder_31...","[Stakeholder_269, Stakeholder_46, Stakeholder_...","[Mitigation_1587, Mitigation_988, Mitigation_1...","[Technology_22, Technology_358, Technology_185...","[Benef_5, Benef_1206, Benef_988]",* The Blueprint outlined addresses the pressin...,"* I recommend implementing the Blueprint, whic...",...,The Blueprint outlined addresses the pressing ...,The Blueprint outlined addresses the pressing ...,The risks associated with the prevailing COVID...,For the Blueprint to be effective in mitigatin...,The implementation of the Blueprint offers a p...,"I recommend implementing the Blueprint, which ...","Stakeholders such as healthcare institutions, ...",'Neighbourhood',The Blueprint lists numerous risks including t...,Neighbourhood
1,PBNCategory_97,Implementing Wastewater Monitoring and Enhance...,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_2, Stakeholder_104...","[Stakeholder_1046, Stakeholder_413, Stakeholde...","[Mitigation_2162, Mitigation_2164, Mitigation_...","[Technology_3066, Technology_3067, Technology_...","[Benef_988, Benef_1206, Benef_5]",The context of this Blueprint revolves around ...,"* The ""Blueprint"" provides a comprehensive gui...",...,The context of this Blueprint revolves around ...,The context of this Blueprint revolves around ...,The risks identified within this blueprint foc...,The mitigation measures laid out in this bluep...,Implementing the mitigation measures laid out ...,"The ""Blueprint"" provides a comprehensive guide...","Wastewater treatment plant operators, research...",Neighbourhood,The blueprint addresses several risks related ...,Neighbourhood
2,PBNCategory_51,Implementing Effective Measures to Combat Cont...,This category of Risks describes a range of is...,"[Stakeholder_7, Stakeholder_562, Stakeholder_9...","[Stakeholder_627, Stakeholder_73, Stakeholder_...","[Mitigation_1853, Mitigation_2467, Mitigation_...","[Technology_22, Technology_23, Technology_2579...","[Benef_1366, Benef_632, Benef_631]",* The context of this blueprint derived from t...,* I strongly recommend implementing the 'Bluep...,...,The context of this blueprint derived from the...,The context of this blueprint derived from the...,The listed risks comprise domestic and intimat...,The blueprint underlines a strategic approach ...,The implementation of this blueprint holds sig...,I strongly recommend implementing the 'Bluepri...,"Government entities, NGOs, and law enforcement...",'Neighbourhood',The risks detailed in 'Blueprint' primarily co...,Neighbourhood
3,PBNCategory_52,Implementing Mental Health Mitigation Strategies,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_68, Stakeholder_29...","[Stakeholder_121, Stakeholder_47, Stakeholder_...","[Mitigation_2128, Mitigation_2315, Mitigation_...","[Technology_22, Technology_146, Technology_280...","[Benef_436, Benef_153, Benef_11]",* The provided blueprint revolves around menta...,"* Based on the analysis, I would highly recomm...",...,The provided blueprint revolves around mental ...,The provided blueprint revolves around mental ...,Several risks are highlighted in the blueprint...,"To tackle these potential threats, a series of...",The implementation of this blueprint or these ...,"Based on the analysis, I would highly recommen...","Healthcare providers, governments, mental heal...",'Neighbourhood',The risks targeted in this blueprint are menta...,Neighbourhood
4,PBNCategory_53,Implementing Collaborative Measures to Combat ...,This category of Risks describes a range of is...,"[Stakeholder_9, Stakeholder_222, Stakeholder_1...","[Stakeholder_73, Stakeholder_1357, Stakeholde

In [184]:
DFFinal.iloc[0].reset_index()

,index,0
0,category,PBNCategory_50
1,title,"Implementing Regular Cleaning, WBE and Quarant..."
2,risks,This category of Risks describes various aspec...
3,subject,"[Stakeholder_68, Stakeholder_9, Stakeholder_31..."
4,owner,"[Stakeholder_269, Stakeholder_46, Stakeholder_..."
5,mitig,"[Mitigation_1587, Mitigation_988, Mitigation_1..."
6,techs,"[Technology_22, Technology_358, Technology_185..."
7,benefs,"[Benef_5, Benef_1206, Benef_988]"
8,summary,* The Blueprint outlined addresses the pressin...
9,details,"* I recommend implementing the Blueprint, whic..."


## Then

@TODO

1. Insert descriptions of Categories / and create new property
2. Create Blueprint_fromCategories - subclass of blueprints
3. Create links (aBP -> stakeholder, aBP->Mitig, aBP->Tech, aBP->Benef, aBP->PBNCategory)
4. Create fields summary, detail, title 
5. Populate